# SVASv


In [1]:
import asyncio
import threading
from typing import Optional

import sirius_sdk
import ipywidgets as widgets


################### P2P connection to Confidential-Storage Provider ###################
PROVIDER: Optional[sirius_sdk.Pairwise] = None
#######################################################

# Init connection to Wallet that maintained in the Cloud
# Off cource developer have alternative - he may maintain Wallet and secrets on other way (KERI library) 
# thanks to Sirius-SDK is friendly to Legacy and flexible due to dependency injection architecture 

CFG = sirius_sdk.Config().setup_cloud(
    server_uri='https://agents.socialsirius.com',
    credentials='s7RxiBlpeNq8k8hrx4vlgjb8XFnGQGxTjIQgk74LgYSYAS4TuR1kZZxJg5MS6b+rUj54zpQ9fWOm3lsdQwI3d2EzIARf+kcCNKU5wKJX9LY=',
    p2p={
        "their_verkey": "FBHVE36ZfixjBwqJWKy45sTRJdt9KDFrqQPL6rExJweN",
        "my_keys": [
          "22UJ1f3nPuJR9y6znPXwGw2xEyzmNRigwbfNoJPraPtY",
          "5JjNff6y8GxWtWxWA5BiKkfZVx4hYQHscwZt7z613CrENWL7xRMehy5MWNy6BqooX73PmDDSso9npNEqsmbXgXCG"
        ]
   }
)

async def establish_connection():
    global PROVIDER
    try:
        async with sirius_sdk.context(CFG):
            did, verkey = await sirius_sdk.DID.create_and_store_my_did()
            manager = sirius_sdk.recipes.InvitationManager(
                me=sirius_sdk.Pairwise.Me(did, verkey),
                my_label='Vault Client'
            )
            invitation = await manager.make_invitation()
            invitation_url = 'https://example.com/invitation' + invitation.invitation_url
            output.append_stdout('\n !!!!!! Copy invitation and PASS to Provider script !!!!!!\n')
            output.append_stdout('_'*64 + '\n')
            output.append_stdout(invitation_url)
            output.append_stdout('\n' + '_'*64)
            
            # Start ti listen connection requests
            output.append_stdout('\nWait for connection requests ...\n')
            PROVIDER = await manager.wait_connection(invitation)
            output.append_stdout('_'*64 + '\n')
            output.clear_output()
            output.append_stdout(f'\n!!! Connection with "{PROVIDER.their.label}" was established !!!!')
            output.append_stdout('\n' + '_'*64)
    except Exception as e:
        output.append_stderr(repr(e))

output = widgets.Output(layout={"width": "600px"})
button = widgets.Button(description="Generate Invitation", button_style='primary')
button_clipboard = widgets.Button(description="Copy to clipboard")

def on_button_clicked(b):
    output.clear_output()
    asyncio.ensure_future(establish_connection())
    
button.on_click(on_button_clicked)
        
display(button, output)


Button(button_style='primary', description='Generate Invitation', style=ButtonStyle())

Output(layout=Layout(width='600px'))

# TODO


In [2]:
import json

from IPython.display import JSON, HTML


assert PROVIDER is not None, "P2P is empty, establish connection with Storage provider"

VAULTS = []
VAULT_ID: Optional[str] = None

# Vaults selection widgets
vaults_dropdown = widgets.Dropdown(description='Vault ID:')
vaults_dropdown.disabled = True
# Log outputs
output_log = widgets.Output(layout={"width": "800px", "border": "1px solid #dddddd", "padding": "10px"})
output_vaults = widgets.Output(layout={"height": "70px", "border": "1px solid #dddddd", "padding": "10px"})
button_list = widgets.Button(description="Query Vaults", button_style='primary')
vboxes = []
cols = [
    # Each tuple contains a column header and a list of items/widgets
    (
        'Operations', 
        [
            widgets.HBox([vaults_dropdown, button_list]),
            output_vaults
        ]
        
    ),
    (
        'Log', 
        [output_log]
    ),
]
for header, data in cols:
    vboxes.append(widgets.VBox([widgets.HTML('<b>%s</b>' % header)] + [
        d if isinstance(d, widgets.Widget) else widgets.HTML(str(d)) for d in data],
    layout=widgets.Layout(border='1px solid #DDDDDD')))
hbox = widgets.HBox(vboxes)

async def query_vaults():
    try:
        async with sirius_sdk.context(CFG):
            protocol = sirius_sdk.aries_rfc.CallerEncryptedDataVault(PROVIDER)
            vaults = await protocol.list_vaults()
            for cfg in vaults:
                js = cfg.as_json()
                VAULTS.append(js)
                output_log.append_stdout(f'\n=========== ID: {js["id"]}=============')
                output_log.append_stdout('\n' + json.dumps(js, indent=2))
            vaults_dropdown.options = [vault['id'] for vault in VAULTS]
            vaults_dropdown.value = VAULTS[0]['id']
            vaults_dropdown.disabled = False
            button_list.disabled = True
    except Exception as e:
        output_log.append_stderr(repr(e))
        
        
async def open_vault(vault_id: str, vault_id_to_close: str = None):
    global VAULT_ID
    await asyncio.sleep(0.1)
    try:
        async with sirius_sdk.context(CFG):
            if vault_id_to_close:
                # Close old session
                output_vaults.append_display_data(HTML(f'<span style="color:red;">UnSelect: [{vault_id_to_close}]</span>'))
            # Select Vault
            VAULT_ID = vault_id
            output_vaults.append_display_data(HTML(f'<span style="color:green;">Selected Vault: [{vault_id}]</span>'))
    except Exception as e:
        output_vaults.append_stderr(repr(e)) 


def clear_ui():
    output_log.clear_output()
    output_vaults.clear_output()
    vaults_dropdown.options = []
    

def on_btn_list_clicked(b):
    VAULTS.clear()
    clear_ui()
    asyncio.ensure_future(query_vaults())
    
    
def on_vaults_dropdown_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        output_vaults.clear_output()
        asyncio.ensure_future(open_vault(vault_id=change['new'], vault_id_to_close=change['old']))
    
    
button_list.on_click(on_btn_list_clicked)
vaults_dropdown.observe(on_vaults_dropdown_change)
display(hbox)


## Create/Load/Remove documents


In [95]:
assert VAULT_ID is not None, "You should to select Vault!"

button_create_doc = widgets.Button(description="> Create Document", button_style='success')
button_remove_doc = widgets.Button(description="> Remove Document", button_style='danger')
button_list_docs = widgets.Button(description="List Documents >", button_style='primary')
checkbox_encrypt_doc = widgets.Checkbox(False, description='Encrypt')
input_create_doc = widgets.Text(
    value='my_document.txt',
    placeholder='Type document name',
    description='Name:  ',
)
input_doc_content = widgets.Textarea(
    value='This is document content',
    placeholder='Fill document content',
    description='Content'
)
input_remove_doc = widgets.Text(
    placeholder='Put here document id or urn',
    description='ID or URN: '
)
output_log_docs = widgets.Output(layout={"width": "800px", "border": "1px solid #eeeeee", "padding": "10px"})
vboxes = []

grid_layout = widgets.GridspecLayout(5, 2)
# fill it in with widgets
grid_layout[0, 0] = input_create_doc
grid_layout[0, 1] = button_create_doc
grid_layout[1, 0] = input_doc_content
grid_layout[2, 0] = checkbox_encrypt_doc
grid_layout[3, 1] = button_list_docs
grid_layout[4, 0] = input_remove_doc
grid_layout[4:, 1] = button_remove_doc
#grid_layout.layout.width = '500px'


cols = [
    # Each tuple contains a column header and a list of items/widgets
    (
        'Doc Operations', 
        [grid_layout]
        
    ),
    (
        'Doc Logs', 
        [output_log_docs]
    ),
]
for header, data in cols:
    vboxes.append(widgets.VBox([widgets.HTML('<b>%s</b>' % header)] + [
        d if isinstance(d, widgets.Widget) else widgets.HTML(str(d)) for d in data],
    layout=widgets.Layout(border='1px solid #DDDDDD')))
hbox = widgets.HBox(vboxes)


async def create_document(name: str, content: bytes):
    try:
        async with sirius_sdk.context(CFG):
            protocol = sirius_sdk.aries_rfc.CallerEncryptedDataVault(PROVIDER)
            
            # Select and Open Vault
            protocol.select(VAULT_ID)
            await protocol.open()
            
            # Create document
            structored_doc = await protocol.create_document(uri=name, category='document')
            jsonable = sirius_sdk.aries_rfc.StructuredDocumentAttach.create_from(src=structored_doc, sequence=0).as_json()
            output_log_docs.append_display_data(HTML("<b style='color: blue;'>Create: Structured document</b>"))
            output_log_docs.append_stdout(json.dumps(jsonable, indent=2))
            
            # Save document
            if checkbox_encrypt_doc.value:
                encrypted_doc = sirius_sdk.aries_rfc.EncryptedDocument(target_verkeys=[PROVIDER.me.verkey])
                encrypted_doc.content = content
                await encrypted_doc.encrypt()
            else:
                encrypted_doc = sirius_sdk.aries_rfc.EncryptedDocument()
                encrypted_doc.content = content
            await protocol.save_document(structored_doc.id, encrypted_doc)
            output_log_docs.append_display_data(HTML("<b style='color: green;'>Document successfully saved!</b>"))
            
            # Load document
            loaded_structured_document = await protocol.load(structored_doc.id)
            jsonable = sirius_sdk.aries_rfc.StructuredDocumentAttach.create_from(src=loaded_structured_document, sequence=0).as_json()
            output_log_docs.append_display_data(HTML("<b style='color: blue;'>Load: Structured document</b>"))
            if loaded_structured_document.doc.encrypted:
                output_log_docs.append_display_data(HTML("<b style='color: red;'>! encrypted !</b>"))
            output_log_docs.append_stdout(json.dumps(jsonable, indent=2))
            
            # Close resources
            await protocol.close()
    except Exception as e:
        output_log_docs.append_stderr(repr(e)) 
        
        
async def remove_document(doc_id: str):
    try:
        async with sirius_sdk.context(CFG):
            protocol = sirius_sdk.aries_rfc.CallerEncryptedDataVault(PROVIDER)
            
            # Select and Open Vault
            protocol.select(VAULT_ID)
            await protocol.open()
            
            # Remove document
            await protocol.remove(uri=doc_id)
            output_log_docs.append_display_data(HTML(f"<b style='color: red;'>Document [{doc_id}] removed</b>"))
            
            # Close resources
            await protocol.close()
    except Exception as e:
        output_log_docs.append_stderr(repr(e)) 
        
        
async def list_documents():
    try:
        async with sirius_sdk.context(CFG):
            protocol = sirius_sdk.aries_rfc.CallerEncryptedDataVault(PROVIDER)
            
            # Select and Open Vault
            protocol.select(VAULT_ID)
            await protocol.open()
            
            # List documents
            indexes = await protocol.indexes()
            docs = await indexes.filter(category='document')
            if docs:
                for i, doc in enumerate(docs):
                    output_log_docs.append_display_data(HTML(f"<b style='color: blue;'>Document [{i}]</b>"))
                    jsonable = sirius_sdk.aries_rfc.StructuredDocumentAttach.create_from(src=doc, sequence=i).as_json()
                    output_log_docs.append_stdout(json.dumps(jsonable, indent=2))
            else:
                output_log_docs.append_display_data(HTML(f"<b style='color: blue;'>Document list is Empty</b>"))
            
            # Close resources
            await protocol.close()
    except Exception as e:
        output_log_docs.append_stderr(repr(e)) 

        
def on_btn_createdoc_clicked(b):
    output_log_docs.clear_output()
    doc_name = input_create_doc.value
    doc_content = input_doc_content.value
    asyncio.ensure_future(create_document(doc_name, doc_content.encode('utf-8')))
    
def on_btn_removedoc_clicked(b):
    output_log_docs.clear_output()
    doc_id = input_remove_doc.value
    asyncio.ensure_future(remove_document(doc_id))
    
    
def on_btn_list_clicked(b):
    output_log_docs.clear_output()
    asyncio.ensure_future(list_documents())
    
    
button_create_doc.on_click(on_btn_createdoc_clicked)
button_remove_doc.on_click(on_btn_removedoc_clicked)
button_list_docs.on_click(on_btn_list_clicked)
display(hbox)

## Upload/Download Streams

In [150]:
button_create_stream = widgets.Button(description="> Create Stream", button_style='success')
button_upload_file = widgets.FileUpload(multiple=False)
output_log_streams = widgets.Output(layout={"width": "800px", "border": "1px solid #eeeeee", "padding": "10px"})
input_create_stream = widgets.Text(
    value='my_stream.bin',
    placeholder='Type stream name',
    description='Name:  ',
)
checkbox_encrypt_stream = widgets.Checkbox(False, description='Encrypt')

vboxes = []

grid_layout = widgets.GridspecLayout(5, 2)
# fill it in with widgets
grid_layout[0, 0] = input_create_stream
grid_layout[0, 1] = button_upload_file
grid_layout[1, 0] = checkbox_encrypt_stream
grid_layout[1, 1] = button_create_stream
#grid_layout.layout.width = '500px'


cols = [
    # Each tuple contains a column header and a list of items/widgets
    (
        'Stream Operations', 
        [grid_layout]
        
    ),
    (
        'Stream Logs', 
        [output_log_streams]
    ),
]
for header, data in cols:
    vboxes.append(widgets.VBox([widgets.HTML('<b>%s</b>' % header)] + [
        d if isinstance(d, widgets.Widget) else widgets.HTML(str(d)) for d in data],
    layout=widgets.Layout(border='1px solid #DDDDDD')))
hbox = widgets.HBox(vboxes)

def show_it(inputs):
    try:
        # output_log_streams.clear_output()
        #output_log_streams.append_stdout(json.dumps(inputs['new'], indent=2))
        value = button_upload_file.value
        output_log_streams.append_stdout(repr(value['kupibilet_order_ru_8131434216 (1).pdf']))
        #output_log_streams.append_stdout(str(list(button_upload_file.value)))
        #content = inputs['new']['value'][0]['content']
    except Exception as e:
        output_log_streams.append_stderr(str(e))

button_upload_file.observe(show_it)
        

display(hbox)